# TF.js Model Conversion — Colab

**Open in Colab:** [colab.research.google.com](https://colab.research.google.com) → File → Upload notebook → select `ml/convert_to_tfjs_colab.ipynb` from your project.

If this repo is on GitHub, you can use:  
`https://colab.research.google.com/github/YOUR_USER/YOUR_REPO/blob/main/ml/convert_to_tfjs_colab.ipynb`

**Steps:** Run all cells. When prompted, upload `model_char.h5`, `model_word.h5`, `charlist.json`. Download the zip, then run `scripts/unpack-tfjs-models.ps1` or manually copy into `public/dhcd/`.

In [ ]:
# Cell 1 — install compatible versions
!pip install "tensorflow==2.15.0" "tensorflowjs==4.17.0" -q

In [ ]:
# Cell 2 — upload your .h5 files (run, then click Choose Files)
from google.colab import files
uploaded = files.upload()  # upload model_char.h5, model_word.h5, and charlist.json (from ml/word/)

In [ ]:
# Cell 3 — convert char model (layers format for loadLayersModel in browser)
!tensorflowjs_converter --input_format keras --output_format tfjs_layers_model model_char.h5 tfjs_char/

In [ ]:
# Cell 4 — convert word model (layers format for loadLayersModel in browser)
!tensorflowjs_converter --input_format keras --output_format tfjs_layers_model model_word.h5 tfjs_word/

In [ ]:
# Cell 5 — sanity check before download
import tensorflow as tf
import numpy as np
import os

# Test char model loads and produces correct output shape
char_model = tf.keras.models.load_model('model_char.h5')
test_input = np.random.rand(1, 32, 32, 1).astype(np.float32)
pred = char_model.predict(test_input)
print(f"✓ Char model output shape: {pred.shape}")  # should be (1, 46)
print(f"✓ Predicted class: {np.argmax(pred)} (confidence {np.max(pred):.3f})")

# Test word model
word_model = tf.keras.models.load_model('model_word.h5')
test_word = np.random.rand(1, 32, 128, 1).astype(np.float32)
pred_word = word_model.predict(test_word)
print(f"✓ Word model output shape: {pred_word.shape}")  # should be (1, timesteps, charset_size)

# Verify TF.js outputs exist and aren't empty
def dir_size(path):
    return sum(os.path.getsize(os.path.join(r, f)) for r, _, files in os.walk(path) for f in files)
char_size = dir_size('tfjs_char')
word_size = dir_size('tfjs_word')
print(f"✓ tfjs_char/ total size: {char_size/1024:.1f} KB")
print(f"✓ tfjs_word/ total size: {word_size/1024:.1f} KB")
assert os.path.exists('tfjs_char/model.json'), "tfjs_char/model.json missing"
assert os.path.exists('tfjs_word/model.json'), "tfjs_word/model.json missing"
print("All checks passed — safe to download")

In [ ]:
# Cell 6 — zip both and download
import zipfile
from pathlib import Path
with zipfile.ZipFile('dhcd_tfjs_all.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
    for d in ['tfjs_char', 'tfjs_word']:
        for f in Path(d).rglob('*'):
            if f.is_file():
                zf.write(f, str(f))
files.download('dhcd_tfjs_all.zip')

## After download

1. Unzip `dhcd_tfjs_all.zip`
2. Copy `tfjs_char/` contents → `public/dhcd/tfjs_model/`
3. Copy `tfjs_word/` contents → `public/dhcd/word_tfjs_model/`
4. Copy `charlist.json` from `ml/word/` → `public/dhcd/` (if not already there)

### Browser smoke test (devtools console)

```javascript
const model = await tf.loadLayersModel('/dhcd/tfjs_model/model.json');
console.log('Char model input shape:', model.inputs[0].shape);   // [null, 32, 32, 1]
console.log('Char model output shape:', model.outputs[0].shape); // [null, 46]
```

If 404 → path wrong. If schema error → add `--output_format tfjs_layers_model` to converter.